In [1]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import os,datetime,warnings,time

import torch
import pandas as pd
import seaborn as sns

from keras.models import Model,Sequential
from keras.layers import Dense,LSTM,Conv2D,Conv2DTranspose,Dropout,GRU,Activation,Add,Concatenate,BatchNormalization,Bidirectional,Embedding,Flatten

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
import string

import re

warnings.filterwarnings('ignore')
sns.set()

C:\Users\Jan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def clean_dataset(X,Y,MAXLEN=15):
    indices = []
    for k in range(len(Y)):
        flag = False
        for yy in Y[k]: 
            if len(yy)>MAXLEN: flag = True
        if not flag: indices.append(k)
    X = [X[k] for k in indices]
    Y = [Y[k] for k in indices]
    return X,Y

In [3]:
bf = pd.read_csv('..\\..\\TF_data\\twitter_data\\train_tweets.csv',index_col='id')

FileNotFoundError: [Errno 2] File b'..\\..\\TF_data\\twitter_data\\train_tweets.csv' does not exist: b'..\\..\\TF_data\\twitter_data\\train_tweets.csv'

In [ ]:
df = bf[['tweet']].copy()

df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'[^a-zA-Z0-9_@ ]','',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\s+',' ',x.lower().strip()))

In [ ]:
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user @user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user @user @user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user @user @user @user @user @user','@user',x))

df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user@user@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user@user@user@user@user@user','@user',x))

In [ ]:
df['tweet'] = df['tweet'].apply(lambda x: x+' <EOL>')

In [ ]:
w,c = np.unique(np.array([w for s in df['tweet'].tolist() for w in s.split(' ')]),return_counts=True)

encode_dict = pd.DataFrame(w,columns=['Word']).reset_index().rename(columns={'index':'Code'}).set_index('Word')
encode_dict['Word'] = encode_dict.index
encode_dict.index = encode_dict.index.rename('idx')
encode_dict['Wordlength'] = encode_dict['Word'].apply(lambda x: len(x))
encode_dict['Freq'] = c

encode_dict.ix[encode_dict[(encode_dict['Freq']==1)].index,'Code']=-1
encode_dict.ix[encode_dict[(encode_dict['Freq']>1)&(encode_dict['Freq']<=100)].index,'Code']=-2
encode_dict.ix[encode_dict[(encode_dict['Freq']>100)&(encode_dict['Freq']<=200)].index,'Code']=-3
encode_dict.ix[encode_dict[(encode_dict['Freq']>200)&(encode_dict['Freq']<=275)].index,'Code']=-4
encode_dict.ix[encode_dict[(encode_dict['Freq']>275)&(encode_dict['Freq']<=350)].index,'Code']=-5
encode_dict.ix[encode_dict[(encode_dict['Freq']>350)&(encode_dict['Freq']<=400)].index,'Code']=-6
encode_dict.ix[encode_dict[(encode_dict['Freq']>400)&(encode_dict['Freq']<=450)].index,'Code']=-7
encode_dict.ix[encode_dict[(encode_dict['Freq']>450)&(encode_dict['Freq']<=475)].index,'Code']=-8
encode_dict.ix[encode_dict[(encode_dict['Freq']>475)&(encode_dict['Freq']<=500)].index,'Code']=-9

In [ ]:
df['tweet_coded'] = df['tweet'].apply(lambda x: [encode_dict.ix[w,'Code'] for w in x.split(' ')])

In [9]:
L = 10
F = 1

X = []
Y = []
for k in df.index.tolist():
    s = df.ix[k,'tweet_coded']
    for kk in range(len(s)-F-L+1):
        X.append(s[kk:kk+L])
        Y.append(s[kk+L+F-1])

In [10]:
ohe = OneHotEncoder(categories='auto')
Y0 = ohe.fit_transform(X)

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)